In [1]:
import sys
sys.executable

'/home/ksmehrab/miniconda/envs/glamm/bin/python'

In [2]:
import cv2
import numpy as np
from PIL import Image
import torch
import os 
from tqdm import tqdm

/home/ksmehrab/miniconda/envs/glamm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup and main code

In [3]:
import sys
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Models/GLAMM')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Datasets')

from run_glamm import segment_image_using_glamm
from pascalpart import get_pascalpart_classes
from utils import read_txt_file, save_to_json
from tqdm import tqdm
import os

---- Initialized tokenizer from: MBZUAI/GLaMM-RefSeg ----


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]


---- Initialized model from: MBZUAI/GLaMM-RefSeg ----
---- Initialized Global Image Encoder (vision tower) from: openai/clip-vit-large-patch14-336 ----


Some weights of the model checkpoint at openai/clip-vit-large-patch14-336 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.6.self_attn.out_proj.weight', 'text_model.encoder.layers.1.layer_norm1.weight', 'text_model.encoder.layers.5.self_attn.v_proj.bias', 'text_model.encoder.layers.0.layer_norm1.weight', 'text_model.encoder.layers.2.mlp.fc1.weight', 'text_model.encoder.layers.11.self_attn.out_proj.bias', 'text_model.encoder.layers.1.layer_norm2.bias', 'text_model.encoder.layers.9.layer_norm2.bias', 'text_model.encoder.layers.10.self_attn.q_proj.weight', 'text_model.encoder.layers.1.self_attn.v_proj.bias', 'text_model.encoder.layers.6.mlp.fc2.weight', 'text_model.encoder.layers.9.layer_norm1.weight', 'text_model.encoder.layers.3.self_attn.k_proj.weight', 'text_model.encoder.layers.10.self_attn.out_proj.bias', 'text_model.encoder.layers.9.mlp.fc2.bias', 'text_model.encoder.layers.7.self_attn.v_proj.weight', 'text_model.encoder.layers.7.layer_norm1.weight', 'te

In [4]:
pascal_image_dir = "/data/Pascal_VOC_2012/VOCdevkit/VOC2012/JPEGImages" # 17125 images
annotations_path= "/data/PartSegmentationDatasets/PascalPart/Annotations_Part"
val_filepath = "/data/PartSegmentationDatasets/PascalPart/val.txt" # 925 images. File contains just the file prefix. Add .jpg extension for images, and .mat extension for annotations
val_filenames = read_txt_file(val_filepath)

model_name = 'glamm'

# This is where crop results are saved 
base_crop_save_dir = "/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM"
if not os.path.exists(base_crop_save_dir):
    os.makedirs(base_crop_save_dir)

# save_dir is where results of initial segmentations have been saved 
save_dir = "/data/VLMGroundingProject/BaselineResults/PascalPart/GLAMM"
if not os.path.exists(save_dir):
    raise ValueError(f"save_dir {save_dir} does not exist. Run the segmentation first.")

already_done = [os.path.splitext(f)[0] for f in os.listdir(base_crop_save_dir)]

req_filenames = [f for f in val_filenames if f not in already_done]

print(f'Already done on {len(already_done)} images. Running on remaining {len(req_filenames)}...')


Already done on 0 images. Running on remaining 925...


In [20]:
def get_bbox_from_mask(mask_path):
    """Extract bounding box from a binary mask."""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    coords = np.where(mask > 0)
    if len(coords[0]) == 0:
        return None
    
    y_min, y_max = coords[0].min(), coords[0].max()
    x_min, x_max = coords[1].min(), coords[1].max()
    
    # Add some padding
    padding = 10
    return [max(0, x_min-padding), max(0, y_min-padding), 
            x_max+padding, y_max+padding]

def crop_image_to_bbox(image_path, bbox):
    """Crop image to bounding box."""
    image = Image.open(image_path)
    return image.crop(bbox)

def run_part_segmentation_on_crops():
    PROMPT_TEMPLATE = "Can you segment the <obj> in this image?"
    
    for filename in tqdm(req_filenames):
        img_filepath = os.path.join(pascal_image_dir, filename+'.jpg')
        annot_filename = filename + '.mat'
        classes_to_detect = get_pascalpart_classes(annot_filename, annotations_path)

        save_dir_for_example = os.path.join(save_dir, filename)
        
        crop_save_dir = os.path.join(base_crop_save_dir, filename)
        os.makedirs(crop_save_dir, exist_ok=True)
        
        for object, parts in classes_to_detect.items():
            object_underscore = object.replace(' ', '_')
            object_mask_path = os.path.join(save_dir_for_example, f'{object_underscore}.png')
            
            if not os.path.exists(object_mask_path):
                continue
                
            # Get bbox from object mask
            bbox = get_bbox_from_mask(object_mask_path)
            if bbox is None:
                continue
                
            # Crop the original image
            cropped_image = crop_image_to_bbox(img_filepath, bbox)
            cropped_image_path = os.path.join(crop_save_dir, f'{object_underscore}_crop.jpg')
            cropped_image.save(cropped_image_path)
            
            # Segment parts in the cropped image
            for part in parts:
                part_prompt = PROMPT_TEMPLATE.replace('<obj>', f'{object} {part}')
                
                part_mask_crop = segment_image_using_glamm(
                    input_image_path=cropped_image_path,
                    prompt_text=part_prompt
                )
                
                part_underscore = part.replace(' ', '_')
                part_mask_crop.save(os.path.join(crop_save_dir, f'{object_underscore}_{part_underscore}_crop.png'))

In [6]:
run_part_segmentation_on_crops()

 22%|██▏       | 202/925 [26:07<1:33:29,  7.76s/it]


KeyboardInterrupt: 

## Evaluation

In [21]:
import sys
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines/Datasets')
sys.path.append('/home/ksmehrab/AttentionGrounding/Baselines')

In [22]:
from pascalpart import get_pascalpart_masks
from utils import read_txt_file
from tqdm import tqdm
import os
from PIL import Image
import numpy as np
import cv2

In [23]:
pascal_image_dir = "/data/Pascal_VOC_2012/VOCdevkit/VOC2012/JPEGImages" # 17125 images
annotations_path= "/data/PartSegmentationDatasets/PascalPart/Annotations_Part"
val_filepath = "/data/PartSegmentationDatasets/PascalPart/val.txt" # 925 images. File contains just the file prefix. Add .jpg extension for images, and .mat extension for annotations
base_pred_dir = "/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM"
orig_pred_dir = "/data/VLMGroundingProject/BaselineResults/PascalPart/GLAMM"

In [24]:
req_filenames = os.listdir(base_pred_dir)

In [25]:
def combine_masks(masks):
    combined_mask = masks[0]
    for i in range(1, len(masks)):
        combined_mask = combined_mask | masks[i]
    return combined_mask

In [26]:
from chainercv.evaluations import calc_semantic_segmentation_confusion
def compute_miou(preds, gts):
    '''
    This function takes two lists as inputs, where:
    preds: [list of 2-D prediction where each pixel corresponds to pixel-label]
    gts: [list of ground-truths in 2-D space, contains -1 value for critical pixels]
    
    returns mean of the iou for all classes
    '''
    confusion = calc_semantic_segmentation_confusion(preds, gts)

    gtj = confusion.sum(axis=1)
    resj = confusion.sum(axis=0)
    gtjresj = np.diag(confusion)
    denominator = gtj + resj - gtjresj
    iou = gtjresj / denominator

    return iou, confusion

In [27]:
def get_glamm_predicted_mask(base_pred_dir, filename_prefix, class_name, is_crop=False):
    """
        Get the predicted mask for a given image and class from GLAMM predictions.
        Args:
            base_pred_dir (str): Base directory where GLAMM predictions are stored.
            filename_prefix (str): Prefix of the filename (without extension).
            class_name (str): Name of the class for which the mask is required.
    """
    class_name = class_name.replace(" ", "_")
    # all pred masks for this filename is in a directory named filename_prefix
    pred_masks_dir = os.path.join(base_pred_dir, filename_prefix)
    # the required mask is in a file named class_name.png. here, class_name can either be the object class (like person) or the object || part class (like person_head)
    if not is_crop:
        req_pred_mask = os.path.join(pred_masks_dir, class_name + ".png")
    else:
        req_pred_mask = os.path.join(pred_masks_dir, class_name+'_crop' + ".png")
    # Load the mask
    mask = Image.open(req_pred_mask).convert("L")  # grayscale

    # Convert to numpy, then 0/1
    mask_np = np.array(mask)
    binary_mask = (mask_np > 0).astype(np.uint8)
    
    return binary_mask


In [32]:
def transform_crop_mask_to_full_image(crop_mask_path, bbox, original_image_shape):
    """Transform a mask from crop coordinates back to full image coordinates."""
    crop_mask = cv2.imread(crop_mask_path, cv2.IMREAD_GRAYSCALE)
    if crop_mask is None:
        print("Crop mask not found at:", crop_mask_path)
        return None
    
    # Create full-size mask
    full_mask = np.zeros(original_image_shape[:2], dtype=np.uint8)
    
    x_min, y_min, x_max, y_max = bbox
    crop_h, crop_w = crop_mask.shape
    
    # Handle potential size mismatches
    actual_h = min(crop_h, y_max - y_min, original_image_shape[0] - y_min)
    actual_w = min(crop_w, x_max - x_min, original_image_shape[1] - x_min)
    
    # Place the crop mask in the full image
    full_mask[y_min:y_min+actual_h, x_min:x_min+actual_w] = crop_mask[:actual_h, :actual_w]
    
    return full_mask

In [33]:
part_ious = []
count_no_obj = 0
for filename in tqdm(req_filenames):
    # gt_obj_masks = []
    # pred_obj_masks = []

    gt_part_masks = []
    pred_part_masks = []

    img_filepath = os.path.join(pascal_image_dir, filename+'.jpg')
    original_image = cv2.imread(img_filepath)
    original_shape = original_image.shape

    annot_filename = filename + '.mat'
    anno_dict = get_pascalpart_masks(annot_filename, annotations_path, images_path=pascal_image_dir)
    orig_pred_dir_for_example = os.path.join(orig_pred_dir, filename)
    
    for obj_name, anno in anno_dict.items():
        object_underscore = obj_name.replace(' ', '_')
        object_mask_path = os.path.join(orig_pred_dir_for_example, f'{object_underscore}.png')

        if not os.path.exists(object_mask_path):
            continue
            
        bbox = get_bbox_from_mask(object_mask_path)
        if bbox is None:
            continue

        parts_masks = anno['parts']
        for part_name, masks in parts_masks.items():
            part_mask = combine_masks(masks)
            gt_part_masks.append(part_mask)
            part_full_name = obj_name + "_" + part_name + '_crop'
            crop_pred_path = os.path.join(base_pred_dir, filename, f'{part_full_name}.png')
            # Transform to full image coordinates
            full_pred_mask = transform_crop_mask_to_full_image(crop_pred_path, bbox, original_shape)
            if full_pred_mask is None:
                skip_image = True
                break
            pred_part_masks.append(full_pred_mask)
        if skip_image:
            break
    if skip_image:
        skip_image = False
        continue

    if len(pred_part_masks) == 0:
        # no part present in this image
        print(f'no parts present in {filename}, skipping...')
        count_no_obj += 1
        continue
    
    # obj_iou_w_bg, obj_confusion = compute_miou(pred_obj_masks, gt_obj_masks)
    # obj_ious.append(obj_iou_w_bg[1]) # only taking the iou of the object class, not the background


    part_iou_w_bg, part_confusion = compute_miou(pred_part_masks, gt_part_masks)
    part_ious.append(part_iou_w_bg[1]) # only taking the iou of the part class, not the background
    

  0%|          | 0/203 [00:00<?, ?it/s][ WARN:0@278.885] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_000811/person_left ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@278.898] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004072/bus_front side_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@278.910] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_001992/person_left ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@278.920] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003105/train_head front side_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@278.929] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/Baselin

Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_000811/person_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004072/bus_front side_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_001992/person_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003105/train_head front side_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003904/cat_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003928/dog_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_001082/bus_front side_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_001874/person_right ear_crop.png
C

[ WARN:0@279.077] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_000123/bird_left eye_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.085] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_001895/dog_right ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.101] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_003874/person_left ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.110] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_002155/dog_left ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.123] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004848/person_l

no parts present in 2008_003858, skipping...
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_003874/person_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_002155/dog_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004848/person_left lower arm_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_004069/car_front side_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_004140/person_right ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_004175/horse_right ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_002035/cow_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrop

[ WARN:0@279.295] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004033/dog_left eye_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.306] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_005038/person_left ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.314] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_003155/aeroplane_left wing_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.322] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_000863/dog_left ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@279.332] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_005156/bic

...

Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_002212/person_right ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_002778/cow_left ear_crop.png
no parts present in 2010_004041, skipping...
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_004219/motorbike_front wheel_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_004994/bird_right eye_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_005344/cat_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_005531/bird_left eye_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_005788/sheep_left ear_crop.png
no parts present in 2010_003531, skipping...
Crop mask not found at: /data/VLMGroun

[ WARN:0@280.249] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_004345/motorbike_front wheel_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.256] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_004399/motorbike_front wheel_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.263] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_004701/cow_right eye_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.290] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_005445/person_right ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.298] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/20

Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004504/cat_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004540/bird_right eye_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004748/car_front license plate_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_004987/motorbike_front wheel_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_006325/cat_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_006341/person_right ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2008_006480/cat_left ear_crop.png
no parts present in 2008_007048, skipping...
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPar

[ WARN:0@280.464] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_001300/train_head front side_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.483] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_001816/bus_front license plate_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.491] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_002221/cow_right ear_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.504] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_002346/person_left eye_crop.png'): can't open/read file: check file path/integrity
[ WARN:0@280.512] global loadsave.cpp:241 findDecoder imread_('/data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2

Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_002348/cat_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_002450/bird_left wing_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_002480/dog_left ear_crop.png
no parts present in 2010_002538, skipping...
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2010_002546/person_left lower arm_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003003/person_right upper arm_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003059/cow_left ear_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/PascalPartCrops/GLAMM/2009_003065/person_left lower arm_crop.png
Crop mask not found at: /data/VLMGroundingProject/BaselineResults/Pasca

In [ ]:
# for filename in tqdm(req_filenames):
#     img_filepath = os.path.join(pascal_image_dir, filename+'.jpg')
#     original_image = cv2.imread(img_filepath)
#     original_shape = original_image.shape
    
#     annot_filename = filename + '.mat'
#     classes_to_detect = get_pascalpart_classes(annot_filename, annotations_path)
    
#     save_dir_for_example = os.path.join(orig_pred_dir, filename)
#     crop_save_dir = os.path.join(save_dir, filename + "_crops")
    
#     file_results = {}
    
#     for object, parts in classes_to_detect.items():
#         object_underscore = object.replace(' ', '_')
#         object_mask_path = os.path.join(save_dir_for_example, f'{object_underscore}.png')
        
#         if not os.path.exists(object_mask_path):
#             continue
            
#         bbox = get_bbox_from_mask(object_mask_path)
#         if bbox is None:
#             continue
        
#         for part in parts:
#             part_underscore = part.replace(' ', '_')
            
#             # Load crop-based prediction
#             crop_pred_path = os.path.join(crop_save_dir, f'{object_underscore}_{part_underscore}_crop.png')
#             if not os.path.exists(crop_pred_path):
#                 continue
            
#             # Transform to full image coordinates
#             full_pred_mask = transform_crop_mask_to_full_image(crop_pred_path, bbox, original_shape)
            
#             # Load ground truth (you'll need to implement this based on your GT format)
#             gt_mask = load_ground_truth_part_mask(filename, object, part)  # You need to implement this
            
#             # Calculate IoU
#             if gt_mask is not None:
#                 iou = calculate_iou(full_pred_mask, gt_mask)
#                 file_results[f'{object}_{part}'] = iou
    
#     results[filename] = file_results